##Custsom Lambda Layers

Use TensorFlow's functional API to build and customize layers using `Lambda` layers

In [2]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten
from tensorflow.keras.models import Model 
from tensorflow.nn import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as k
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import numpy as np

##Prep the data 

Split data into training and test sets 

In [5]:
(train_data, train_label), (test_data, test_label) = mnist.load_data()

train_data = train_data / 255.0
test_data = test_data / 255.0
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

#Build the model

In [11]:
input = Input(shape=(28, 28), name='input_layer')
flatten = Flatten(name='flatten_layer')(input)

layer_with_activation = Dense(128, activation=relu, name='layer_with_activation')(flatten)
output_layer_with_activation = Dense(10, activation=softmax, name='output_layer_with_activation')(layer_with_activation)

model_with_activation = Model(inputs=input, outputs=output_layer_with_activation)

model_with_activation.compile(optimizer=RMSprop(), loss = categorical_crossentropy, metrics=['acc'])

model_with_activation.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28)]          0         
                                                                 
 flatten_layer (Flatten)     (None, 784)               0         
                                                                 
 layer_with_activation (Dens  (None, 128)              100480    
 e)                                                              
                                                                 
 output_layer_with_activatio  (None, 10)               1290      
 n (Dense)                                                       
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


##Train the model

In [14]:
model_with_activation.fit(train_data, train_label, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2533 - acc: 0.9265
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1194 - acc: 0.9649
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0875 - acc: 0.9738
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0714 - acc: 0.9792
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0606 - acc: 0.9825


##Evaluate the model

In [15]:
model_with_activation.evaluate(test_data, test_label)

313/313 [==============================] - 1s 2ms/step - loss: 0.0935 - acc: 0.9744


[0.09349779039621353, 0.974399983882904]

##Build model without activation


In [20]:
layer_without_activation = Dense(128, name='layer_without_activation')(flatten)
output_layer_without_activation = Dense(10, activation=softmax, name='output_layer_without_activation')(layer_without_activation)

model_without_activation = Model(inputs=input, outputs=output_layer_without_activation)

model_without_activation.compile(optimizer=RMSprop(), loss = categorical_crossentropy, metrics=['acc'])

model_without_activation.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28)]          0         
                                                                 
 flatten_layer (Flatten)     (None, 784)               0         
                                                                 
 layer_without_activation (D  (None, 128)              100480    
 ense)                                                           
                                                                 
 output_layer_without_activa  (None, 10)               1290      
 tion (Dense)                                                    
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [21]:
model_without_activation.fit(train_data, train_label, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3612 - acc: 0.8982
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3065 - acc: 0.9171
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2951 - acc: 0.9190
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2890 - acc: 0.9204
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2853 - acc: 0.9219


In [22]:
model_without_activation.evaluate(test_data, test_label)

313/313 [==============================] - 1s 2ms/step - loss: 0.2818 - acc: 0.9214


[0.2818007469177246, 0.9214000105857849]

You can see that there is about a 7-8% drop in performance due to the removal of the `relu` activation

##Custom Lambda Layer Model 

This model is build with a custom `Lambda` layer where the internal function will pass on the absolute value of `x`

In [23]:
model_lambda = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128),
      tf.keras.layers.Lambda(lambda x: tf.abs(x)),
      tf.keras.layers.Dense(10, activation='softmax')
])

In [24]:
model_lambda.compile(optimizer=RMSprop(), loss = categorical_crossentropy, metrics=['acc'])

model_lambda.fit(train_data, train_label, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2193 - acc: 0.9372
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0958 - acc: 0.9719
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0682 - acc: 0.9800
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0543 - acc: 0.9838
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0436 - acc: 0.9868


In [25]:
model_lambda.evaluate(test_data, test_label)

313/313 [==============================] - 1s 4ms/step - loss: 0.0837 - acc: 0.9771


[0.08368004858493805, 0.9771000146865845]

##Custom relu activation

This model will use a custom `relu` function that will return either `x` if it is greater than `0` or `0` and pass on that value to the following layer

In [35]:
def myRelu(x):
  return k.maximum(0.4, x)

In [36]:
model_lambda_func = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28,28)),
      tf.keras.layers.Dense(128),
      tf.keras.layers.Lambda(myRelu),
      tf.keras.layers.Dense(10, activation='softmax')
])

model_lambda_func.compile(optimizer = RMSprop(), loss = categorical_crossentropy)

model_lambda.fit(train_data, train_label, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0060 - acc: 0.9983
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0052 - acc: 0.9985
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0043 - acc: 0.9988
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0036 - acc: 0.9990
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0030 - acc: 0.9991


In [37]:
model_lambda.evaluate(test_data, test_label)

313/313 [==============================] - 1s 2ms/step - loss: 0.1284 - acc: 0.9782


[0.12844979763031006, 0.9782000184059143]